In [1]:
import pandas as pd
import numpy as np
import time
import math

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

from IPython.display import display

file_column_mappings = {
    "orders_after_2018-05-01_data_000": ["order_id", "map_attributes_imagery_dates", 
                                         "map_attributes_selected_imagery_date", "map_attributes_selected_version", 
                                         "map_attributes_source", "hyperion_attributes_allows_mfr", 
                                         "hyperion_attributes_is_mfr", "hyperion_attributes_is_prospect", 
                                         "hyperion_attributes_property_type", "hyperion_attributes_user_first_capture",
                                         "image_score", "deliverable_id", "deliverable_changed", "created_at", 
                                         "state", "site_id", "site_name", "sneaky", "local", "practice", 
                                         "priority", "roof_overlay", "failure_reason", "skip_markups"],
    
    "orders_after_2018-05-01_images_000": ["order_id", "score", "kind", "location", "distance", 
                                           "gps_accuracy", "make", "model"],
    
    "orders_after_2018-05-01_roof_visibility_000": ["order_id", "roof_visibility_attribute_ids"],
    
    "orders_after_2018-05-01_puborders_000": ["order_id", "order_state", "order_created_at", "order_updated_at", 
                                              "original_order_id", "order_complexity", "order_image_score", 
                                              "order_image_score_name", "priority", "order_deliverable", 
                                              "order_most_recent_failure_reason", "site_id", "order_local", 
                                              "slayer_job_id", "order_site_name", "sneaky", "order_skip_markups", 
                                              "number_of_stories", "e_number_of_stories_name", 
                                              "complexity_attributes_total_roof_facets_above_4_sqft", 
                                              "complexity_attributes_total_roof_facets", 
                                              "complexity_attributes_roof_square_footage", 
                                              "complexity_attributes_walls_square_footage", 
                                              "complexity_attributes_is_mfr", "map_attributes_source", 
                                              "slayer_attributes_allows_mfr", "slayer_attributes_is_mfr",
                                              "slayer_attributes_user_first_capture", "poser_release_number", 
                                              "structure_type", "structure_type_name", "poser_solve_status", 
                                              "hover_now", "first_qa_datetime", "first_qa_date", "models_reprocessed",
                                              "order_needs_to_be_reprocessed_flag", "special_request", "original_roof",
                                              "first_qa_for_state", "last_qa_for_state", "first_qa_email", 
                                              "first_qa_site_name", 
                                              "first_qa_majority_of_doors_within_95_percent_of_standard_sizes", 
                                              "first_qa_map_overlay_accurate", "first_qa_scale_within5_percent", 
                                              "first_qa_should_we_unfail_the_order", "qa_comments", "orthotag_list",
                                              "level_list", "pitch_estimate", "roof_estimate", 
                                              "labeler_roof_estimate", "markup_verifier_roof_estimate", 
                                              "markup_verifier_10_percent", "pitch_actual", "scaling_method",
                                              "visible_on_ortho_map", "door_areal_scale_diff", 
                                              "doors_used_for_scaling", "door_scale_deviation", "orthotag_tree_pass",
                                              "orthotag_poor_image_quality", "orthotag_none", 
                                              "orthotag_full_occlusion", "orthotag_not_on_map", 
                                              "orthotag_tree_occlusion", "orthotag_tree_fail", 
                                              "orthotag_missing_geometry", "orthotag_shadow", 
                                              "poser_solved_image_count", "poser_solved", "failed_complete_timeline", 
                                              "failed_complete_progression", "most_recent_failure_datetime", 
                                              "completed_count", "failed_count", "failed_or_complete_name_first",
                                              "failed_or_complete_name_second", "failed_or_complete_name_third", 
                                              "failed_or_complete_name_fourth", "failed_or_complete_name_fifth",
                                              "failed_or_complete_datetime_first", 
                                              "failed_or_complete_datetime_second", 
                                              "failed_or_complete_datetime_third", 
                                              "failed_or_complete_datetime_fourth",
                                              "failed_or_complete_datetime_fifth", "number_of_images",
                                              "average_image_score", "image_count_deleted", "image_count_0_sides",
                                              "image_count_1_side", "image_count_2_sides", "image_count_unscored", 
                                              "first_completion", "most_recent_completion", 
                                              "unfail_premium_order_flag", "turn_around_time_total", 
                                              "turn_around_time_first_completion", "total_resolution_time_total", 
                                              "total_resolution_time_first_completion", "human_bpo_time_waiting_total",
                                              "human_bpo_time_waiting_first_completion", "turn_around_time_wasted",
                                              "human_bpo_time_wasted", "labeling_time_total", 
                                              "labeling_time_first_completion", "markup_verifying_time_total",
                                              "markup_verifying_time_first_completion", "model_building_time_total",
                                              "model_building_time_first_completion", "modeling_time_total", 
                                              "modeling_time_first_completion", "model_completing_time_total", 
                                              "model_completing_time_first_completion", "web_segmenting_time_total",
                                              "web_segmenting_time_first_completion", "model_segmenting_time_total",
                                              "model_segmenting_time_first_completion", "verifying_time_total", 
                                              "verifying_time_first_completion", "texturing_time_total", 
                                              "texturing_time_first_completion", "qa_time_total", 
                                              "openings_marking_time_total", "openings_marking_time_first_completion",
                                              "vps_marking_time_total", "vps_marking_time_first_completion", 
                                              "primitive_marking_time_total", 
                                              "primitive_marking_time_first_completion"],
    
    "orders_after_2018-06-10_pubtasks_000": ["order_id", "state_transition_id", "transition_type", "transition_id",
                                             "image_id", "namespace", "event", "unlock_indicator", "from_state", 
                                             "to_state", "trigger_user_id", "seconds_in_state", 
                                             "transition_created_at", "image_markup_corrected_openings", 
                                             "image_markup_corrected_vps", "image_markup_corrected_primitive", 
                                             "image_markup_verify_result_reason_id_vps", "image_markup_verify_result_reason_id_openings", 
                                             "image_markup_verify_result_reason_id_primitive", "first_completion", 
                                             "most_recent_completion", "first_failure", "most_recent_failure", 
                                             "completion_time", "completion_count", "previous_completion_time", 
                                             "previous_completion_count", "total_completion_count", 
                                             "last_order_state_transition_id_of_type_before_complete", 
                                             "last_state_transition_of_type_before_complete_indicator", 
                                             "original_order_id", "order_current_state", "deliverable_name", 
                                             "order_image_score", "image_score_name", "sneaky", "order_complexity",
                                             "orders_most_recent_failure_reason_name", "skip_markups", 
                                             "complexity_attributes_is_mfr", "slayer_attributes_allows_mfr", 
                                             "slayer_attributes_is_mfr", "slayer_attributes_user_first_capture", 
                                             "poser_release_number", "structure_type", "poser_solve_status", "hover_now", 
                                             "map_attributes_source", "poser_solved_indicator", "order_site_name", 
                                             "modeler_email", "modeler_site_name", "modeler_roles", "resource_type", 
                                             "modeler_proficiency_level", "waiting_indicator", "unfail_premium_order_indicator"],
}

file_name_name_mapping = {
    #"orders_after_2018-05-01_data_000": "orders",
    #"orders_after_2018-05-01_images_000": "images",
    #"orders_after_2018-05-01_roof_visibility_000": "roof_visibility",
    #"orders_after_2018-06-10_puborders_000": "puborders",
    "orders_after_2018-06-10_pubtasks_000": "pubtasks",
}

data_loc = '/Users/tommy/workspace/hackerthondata/data/'


In [2]:
def load_data():
    print("Loading Data... ")
    data = {}
    for file_name, column_names in file_column_mappings.items():
        nice_name = file_name_name_mapping.get(file_name, None)
        if nice_name:
            print("Loading %s"%(nice_name))
            file_name = data_loc+file_name
            data[nice_name] = pd.read_csv(file_name,
                                          sep='|',
                                          index_col=False, 
                                          names=column_names
                                        )
    print("Data Loaded!")
    return data

raw_data_dic = load_data()
raw_data = raw_data_dic['pubtasks']

Loading Data... 
Loading pubtasks


/Users/tommy/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (13,14,25) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Data Loaded!


In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

for data_name, df in raw_data.items():
    print("\n\n"+data_name + ": ")
    display(df.head())

In [4]:
print("Sorting Data...")
pubtasks = raw_data
pubtasks["transition_created_at"] = pd.to_datetime(pubtasks["transition_created_at"], utc=True)
pubtasks.index = pubtasks["transition_created_at"] 
pubtasks.sort_index(ascending=False, inplace=True)
print("Sorting Complete")

Sorting Data...
Sorting Complete


In [5]:
order_transitions = pubtasks[pubtasks["transition_type"] == "order"]

In [10]:
test_task = order_transitions[0:500]

In [13]:
test_task.to_pickle("small_task.pkl")

In [15]:
read = pd.read_pickle("small_task.pkl")

In [16]:
print(read.shape)

(500, 57)


In [17]:
import time

def order_counts_by_minute(data_arr):

    print("Processing...")
    start = time.time()

    minutes = []
    states_per_minute = []

    current_order_states = {}

    orders_seen = set()

    open_orders = set()

    completed_orders = set()

    current_minute = int(int(time.mktime(data_arr[0].transition_created_at.timetuple())) / 60)

    # Ignore these orders because for some reason they don't get 'closed'
    bad_orders = set([766938, 766942, 766943, 766944, 766946, 766951, 766952,
                          766953, 766954, 766955, 766957, 766959, 766964, 766971])

    i = 0
    dont_skip = True
    for row in data_arr:
        from_state = row.from_state
        to_state = row.to_state
        order_id = row.order_id

        if order_id not in completed_orders:
            if dont_skip is True or order_id == 766938:
                minute = int(int(time.mktime(row.transition_created_at.timetuple())) / 60)
                vals = [from_state, to_state, order_id, minute]

                #print(vals)
                orders_seen.add(order_id)
                open_orders.add(order_id)

                if minute != current_minute:
                    order_state_snapshot = {}
                    for state, orders in current_order_states.items():
                        order_state_snapshot[state] = len(orders)

                    order_state_snapshot["minute"] = current_minute

                    minutes.append(current_minute)
                    states_per_minute.append(order_state_snapshot)

                    current_minute = minute



                if to_state and not isinstance(to_state, float):
                    if to_state not in current_order_states:
                        current_order_states[to_state] = set()

                    # Remove the order from all states it may be in
                    for order_state_name in current_order_states.keys():
                        if order_id in current_order_states[order_state_name]:
                            current_order_states[order_state_name].remove(order_id)

                    if to_state == "new":
                        completed_orders.add(order_id)
                        if order_id in open_orders:
                            open_orders.remove(order_id)
                    else:
                        # Add the from state, as we're moving backwards!
                        if from_state not in current_order_states:
                            current_order_states[from_state] = set()
                        current_order_states[from_state].add(order_id)


    print("Complete")
    end = time.time()
    print(end - start)
    
    return states_per_minute, orders_seen, open_orders

In [21]:
order_transitions_slice = order_transitions[order_transitions["sneaky"] == 'f']
order_transitions_slice = order_transitions_slice[["transition_created_at", "order_id", "from_state", "to_state"]]
# Convert it to a list once as RAM is cheaper than my time :-)
order_transitions_arr = list(order_transitions_slice.itertuples())

states_per_minute, orders_seen, open_orders = order_counts_by_minute(order_transitions_arr)

Processing...
25524186
25524186
25524186
25524186
25524186
25524186
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524185
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524184
25524183
25524183
25524183
25524183
25524183
25524183
25524

25524166
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524165
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
25524164
2

25524143
25524143
25524143
25524143
25524143
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524142
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
25524141
2

25524119
25524119
25524119
25524119
25524119
25524119
25524119
25524119
25524119
25524119
25524119
25524119
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524118
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524117
25524116
25524116
25524116
25524116
25524116
25524116
25524116
25524116
25524116
25524116
25524116
2

25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524098
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524097
25524096
25524096
25524096
25524096
25524096
25524096
25524096
25524096
25524096
25524096
25524096
25524096
2

...

25523432
25523432
25523432
25523432
25523432
25523432
25523432
25523432
25523432
25523432
25523432
25523432
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523431
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523430
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523429
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523428
25523427
25523427
25523427
2

25523376
25523376
25523376
25523376
25523376
25523376
25523376
25523376
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523375
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523374
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
25523373
2

25523335
25523335
25523335
25523335
25523335
25523335
25523335
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523334
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523333
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523332
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
25523331
2

25523220
25523220
25523220
25523220
25523220
25523220
25523220
25523220
25523220
25523220
25523220
25523220
25523220
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523219
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523218
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523217
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523216
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523215
25523214
25523214
25523214
25523214
2

KeyboardInterrupt: 

In [19]:
print(states_per_minute[6])

{'waiting_primitives_processing': 0, 'waiting_image_processing': 0, 'primitives_processing': 0, 'image_processing': 0, 'waiting_verifying': 7, 'waiting_web_segmenting': 6, 'correcting_labels': 0, 'web_segmenting': 2, 'new': 0, 'waiting_modeling': 8, 'model_segmenting': 7, 'complete': 1, 'waiting_pro_measurements_processing': 0, 'waiting_cv_processing': 0, 'waiting_model_segmenting': 2, 'waiting_qa': 0, 'cv_processing': 0, 'pro_measurements_processing': 0, 'modeling': 8, 'minute': 25524180, 'building_processing': 3, 'waiting_markup': 1, 'verifying': 9, 'texturing': 2, 'labeling': 1, 'waiting_zulu_processing': 0, 'extracting_measurements': 2, 'waiting_texturing': 1, 'waiting_labeling': 2, 'zulu_processing': 0, 'waiting_markup_verifying': 0, 'exporting_3d_formats': 1}
